In [0]:
# Importing neccessary packages
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import json
from utils import add_regularization, get_data_generators, plot_model_results
from Models import TrainingCheckpoint, ModelTrain

In [0]:
NUM_CLASSES = 11
with open('classes_list.json', 'r') as fp:
    classes_list = json.load(fp)

In [0]:
# Data augmentation for regularization and adding some extra training samples

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet.preprocess_input,
    rescale=1.0/255.0,
    horizontal_flip=True, 
    zoom_range=0.2,
    shear_range=0.2
)

In [0]:
TARGET_DIM = 300
BATCH_SIZE = 32

In [0]:
train_generator, validation_generator = get_data_generators(datagen, TARGET_DIM, BATCH_SIZE)

Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.


In [0]:
train_generator.class_indices

{'Bread': 0,
 'Dairyproduct': 1,
 'Dessert': 2,
 'Egg': 3,
 'Friedfood': 4,
 'Meat': 5,
 'NoodlesPasta': 6,
 'Rice': 7,
 'Seafood': 8,
 'Soup': 9,
 'VegetableFruit': 10}

In [0]:
validation_generator.class_indices

{'Bread': 0,
 'Dairyproduct': 1,
 'Dessert': 2,
 'Egg': 3,
 'Friedfood': 4,
 'Meat': 5,
 'NoodlesPasta': 6,
 'Rice': 7,
 'Seafood': 8,
 'Soup': 9,
 'VegetableFruit': 10}

In [0]:
# Lets try the Resnet50 architecture

base_model = tf.keras.applications.resnet.ResNet50(
    include_top=False, 
    weights='imagenet', 
    input_shape=(TARGET_DIM, TARGET_DIM, 3)
)

94773248/94765736 [==============================] - 1s 0us/step


In [0]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 306, 306, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 150, 150, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 150, 150, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [0]:
print('Layers in Resnet50: ' + str(len(base_model.layers)))

Layers in Resnet50: 175


In [0]:
preds = base_model.output
preds = tf.keras.layers.GlobalAveragePooling2D()(preds)
preds = tf.keras.layers.Dense(1024, activation=tf.nn.relu)(preds)
preds = tf.keras.layers.BatchNormalization()(preds)
preds = tf.keras.layers.Dense(512, activation=tf.nn.relu)(preds)
preds = tf.keras.layers.BatchNormalization()(preds)
preds = tf.keras.layers.Dense(256, activation=tf.nn.relu)(preds)
preds = tf.keras.layers.BatchNormalization()(preds)
preds = tf.keras.layers.Dense(128, activation=tf.nn.relu)(preds)
preds = tf.keras.layers.Dense(11, activation=tf.nn.softmax)(preds)

In [0]:
model = tf.keras.models.Model(base_model.input, preds)

In [0]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 306, 306, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 150, 150, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 150, 150, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [0]:
resnet50 = ModelTrain(model, TARGET_DIM, BATCH_SIZE)
resnet50.freeze_layers(9)

In [0]:
resnet50.model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 306, 306, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 150, 150, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 150, 150, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [0]:
resnet50.model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['acc']
)

In [0]:
# Lets define checkpoint for model saving
filepath="./models/resnet50-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

training_checkpoint = TrainingCheckpoint()

csv_logger = tf.keras.callbacks.CSVLogger(filename='./logs/resnet50_training.csv', append=True)

In [0]:
#Let's train the model 60 epochs as it will take a lot of time

resnet50.model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[checkpoint, csv_logger, training_checkpoint],
    epochs=60
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/60
308/308 [==============================] - ETA: 0s - loss: 2.1742 - acc: 0.2522
Epoch 00001: val_acc improved from -inf to 0.17231, saving model to ./drive/My Drive/food_11_weights/resnet50-01-0.17.hdf5
308/308 [==============================] - 288s 934ms/step - loss: 2.1742 - acc: 0.2522 - val_loss: 2.5639 - val_acc: 0.1723
Epoch 2/60
308/308 [==============================] - ETA: 0s - loss: 1.9741 - acc: 0.3019
Epoch 00002: val_acc did not improve from 0.17231
308/308 [==============================] - 286s 927ms/step - loss: 1.9741 - acc: 0.3019 - val_loss: 6.5842 - val_acc: 0.1650
Epoch 3/60
308/308 [==============================] - ETA: 0s - loss: 1.8788 - acc: 0.3385
Epoch 00003: val_acc improved from 0.17231 to 0.17640, saving model to ./drive/My Drive/food_11_weights/resnet50-03-0.18.hdf5
308/308 [==============================] - 286s 929ms/step - loss: 1.8788 - acc: 0.3385 - val_loss: 7.

### Results

Training accuracy: 0.5326<br>
Training loss: 1.3332 <br>

Validation accuracy: 0.52307<br>
Validation loss: 1.4663<br>

Avoidable bias: 0.4674<br>
Variance: 0.00953<br>

Avoidable bias is higher than variance, so model needs to fit on training data.